In [127]:

import pandas as pd
import numpy as np
import string, textblob, re
import pickle
import nltk 

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from unidecode import unidecode
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras import layers, models, optimizers
from keras.preprocessing import text, sequence
import matplotlib.pyplot as plt

In [128]:

import chardet
import pandas as pd

with open('./data/out_Tax.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large


df = pd.read_csv('./data/out_Tax.csv', encoding=result['encoding'])

In [154]:
df.head()

,Date,Title,Numac,Link FR,Link NL,Text
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,belgielexbe kruispuntbank wetgeving raad state...
1,1/16/2020,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE ...,2020010044,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,belgielexbe kruispuntbank wetgeving raad state...
2,1/16/2020,SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 20...,2020040052,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,belgielexbe kruispuntbank wetgeving raad state...
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,belgielexbe kruispuntbank wetgeving raad state...
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,belgielexbe kruispuntbank wetgeving raad state...


In [153]:
data_text = df[['Text']]
df['Text'][14]

'belgielexbe kruispuntbank wetgeving raad state eli navigatie systeem via europese identificatiecode wetgeving http wwwejusticejustfgovbeelibesluitstaatsblad einde eerste woord laatste woord publicatie numac brussels hoofdstedelijk gewest januari ministerieel besluit vaststelling modellen formulier bedoeld artikel ss besluit brusselse hoofdstedelijke regering januari vaststelling modaliteiten gunstregime toepasselijk successierechten overdracht familiale ondernemingen familiale vennootschappen minister brusselse hoofdstedelijke regering belast financien begroting gelet wetboek successierechten artikel ss ingevoegd ordonnantie december houdende tweede deel fiscale hervorming gelet besluit brusselse hoofdstedelijke regering januari vaststelling modaliteiten gunstregime toepasselijk successierechten overdracht familiale ondernemingen familiale vennootschappen artikel gelet gelijkekansentest uitgevoerd toepassing artikel ordonnantie oktober invoering gelijkekansentest overwegende artikel b

In [131]:
def remove_accented_chars(text):
    text = unidecode(text)
    return text

def remove_numbers(text): 
    result = re.sub(r'\d+', '', text) 
    return result

def remove_slash_with_space(text): 
    return text.replace('\\', " ")

def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

def text_lowercase(text): 
    return text.lower()  

def remove_whitespace(text): 
    return  " ".join(text.split()) 

def remove_stopwords(text):
    nltk.download('stopwords')
    stop_words = stopwords.words('dutch')
    nstopword = ['bis', 'NL', 'FR']
    stop_words.extend(nstopword)
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    return ' '.join(filtered_text)

def stem_words(text): 
    stemmer = PorterStemmer() 
    word_tokens = word_tokenize(text) 
    stems = [stemmer.stem(word) for word in word_tokens] 
    return ' '.join(stems)

def lemmatize_words(text): 
    lemmatizer = WordNetLemmatizer() 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    return ' '.join(lemmas) 


In [140]:
# Perform preprocessing

def perform_preprocessing(text):
    
    text = remove_accented_chars(text)
    text = remove_numbers(text)
    text = remove_stopwords(text)
    text = text_lowercase(text)
    text = remove_slash_with_space(text)
    text = remove_punctuation(text)
    # text = stem_words(text)
    text = remove_whitespace(text)
    text = lemmatize_words(text)
    return text

text_sample= df['Text'][4]
text_sample

'belgielexbe kruispuntbank wetgeving raad state eli navigatie systeem via europese identificatiecode wetgeving http wwwejusticejustfgovbeelibesluitstaatsblad einde eerste woord laatste woord publicatie numac brussels hoofdstedelijk gewest januari ministerieel besluit vaststelling modellen formulier bedoeld artikel ss besluit brusselse hoofdstedelijke regering januari vaststelling modaliteiten gunstregime toepasselijk successierechten overdracht familiale ondernemingen familiale vennootschappen minister brusselse hoofdstedelijke regering belast financien begroting gelet wetboek successierechten artikel ss ingevoegd ordonnantie december houdende tweede deel fiscale hervorming gelet besluit brusselse hoofdstedelijke regering januari vaststelling modaliteiten gunstregime toepasselijk successierechten overdracht familiale ondernemingen familiale vennootschappen artikel gelet gelijkekansentest uitgevoerd toepassing artikel ordonnantie oktober invoering gelijkekansentest overwegende artikel b

In [141]:
df['Text'] =  perform_preprocessing(text_sample)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Malika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [148]:
df['Text']

0      belgielexbe kruispuntbank wetgeving raad state...
1      belgielexbe kruispuntbank wetgeving raad state...
2      belgielexbe kruispuntbank wetgeving raad state...
3      belgielexbe kruispuntbank wetgeving raad state...
4      belgielexbe kruispuntbank wetgeving raad state...
                             ...                        
764    belgielexbe kruispuntbank wetgeving raad state...
765    belgielexbe kruispuntbank wetgeving raad state...
766    belgielexbe kruispuntbank wetgeving raad state...
767    belgielexbe kruispuntbank wetgeving raad state...
768    belgielexbe kruispuntbank wetgeving raad state...
Name: Text, Length: 769, dtype: object

In [144]:
len(word_tokenize(df['Text'][4]))



249